In [ ]:
!pip install modAl

# Active transfer learning



pool is a list of data that it hasn't been labled yet.
labeled_data is a list of data that has been labeled

# Experiment Setting

In [ ]:
topic_num = 2 # 2 for Ferguson, 8 for Sydney seige and 0 for Charlie Hebdo
# 'tweetBERT'
reps = ['tweetBERT'] # or BERT or Glove
models = ['knn3','knn5','lda','gp','qda']
stratefies = ['lc','Random'] 

GITHUB_DATA_URL = "https://github.com/parsafarinnia/Active-Learning-for-Rumor-Identification-on-Social-Media"
!git clone "https://github.com/parsafarinnia/Active-Learning-for-Rumor-Identification-on-Social-Media"
DATA_PATH = '/content/Active-Learning-for-Rumor-Identification-on-Social-Media'

Cloning into 'Active-Learning-for-Rumor-Identification-on-Social-Media'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 63 (delta 17), reused 33 (delta 4), pack-reused 0
Unpacking objects: 100% (63/63), done.


# Setup

In [ ]:
import tensorflow as tf
import torch
import os
import pickle
import pandas as pd
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import matplotlib.pyplot as plt
% matplotlib inline
import json
import seaborn as sns
from tensorflow import keras
from sklearn.metrics import matthews_corrcoef
!pip install modAL
tf.random.set_seed(42)
np.random.seed(42)
def save_result(h,dest_add):
  with open(dest_add, 'wb') as f:
    np.save(f, h)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Importing Data
getting a test and train data set

In [ ]:

# 8 Sydney

def load_data(path=DATA_PATH+"/all_topics.p",topic_num=0):
  all_topics = pickle.load( open(path, "rb" ) )
  data=pd.DataFrame.from_dict(all_topics[0]).T
  data['topic']=0
  for i in range(1,9):
    topic = pd.DataFrame.from_dict(all_topics[i]).T
    if i == 3: print('hi')
    topic['topic'] = i
    data = pd.concat([data,topic],axis=0)
  map = {"rumours":1,"non-rumours":0}
  data=data.replace({'tag':map})
  data  
  test_df = data[data['topic']==topic_num]
  train_df = data[data['topic']!=topic_num]
  train_df.head()

  data2=data.reset_index()
  start_test=data2[data2['topic']==topic_num].iloc[0]
  # end_test=data2[data2['topic']==0].iloc[-1]
  # start_train =data2[data2['topic']==1].iloc[0]
  # end_train = start_test=data2[data2['topic']==8].iloc[-1]
  def pd_iter_func(df,topic):
      for row in df.itertuples():
          # Define your criteria here
          if row.topic ==topic:
              return row
  start_test=pd_iter_func(data2,topic_num).Index
  if topic_num==8: 
    end_test = len(data2)
  else:
    end_test = pd_iter_func(data2,topic_num+1).Index-1

  train_start = pd_iter_func(data2,topic_num).Index
  # train_end = len(data2)
  # print(train_start,',',train_end)
  train_sentences = train_df.text.values
  test_sentences = test_df.text.values
  train_labels = train_df.tag.values
  test_labels = test_df.tag.values
  len(train_labels)
  test_labels = test_df.tag.values
  train_start=start_test
  train_end=end_test
  print(start_test,end_test)
  return train_start,train_end,test_labels,test_sentences

train_start,train_end,test_labels,test_sentences = load_data(topic_num=topic_num)

hi
2093 3235


# ACTIVE LEARNING LOOP

## POOL and Labeled data and initial model defenition

### Models


#### ada boost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()

#### MLP


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
def create_keras_model_berts():
  model = Sequential()
  model.add(keras.Input(shape=(768,),name="source"))
  model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.3))
  # model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
def create_keras_model_GLOVE():
  model = Sequential()
  model.add(keras.Input(shape=(200,),name="source"))
  model.add(Dense(128, activation='relu'))
  # model.add(Dropout(0.3))
  # model.add(Dense(10, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(2, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from functools import partial


#### SVM

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import svm
clf_svm = svm.SVC(probability=True)


In [ ]:
from sklearn import svm
def svm_model(data_train,Y_train):
  clf = svm.SVC(probabilty=True)
  clf=clf.fit(data_train, Y_train)
  return clf


#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier(max_depth=1000, random_state=0)


## Running loop

### Run_multy_sklearn

Runs multiple expeiments, averaging the results

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
def get_model(model_name):
  if model_name =="Ada":
    clf = AdaBoostClassifier()
  if model_name =="svm":
    clf=svm.SVC(probability=True)
  if model_name =="rf":
    clf=RandomForestClassifier(max_depth=1000, random_state=0)
  if model_name =="LR":
    clf=LogisticRegression(random_state=0)
  if model_name =="knn3":
    clf=KNeighborsClassifier(n_neighbors=3)
  if model_name =="knn5":
    clf=KNeighborsClassifier(n_neighbors=5)
  if model_name =="gp":
     clf=GaussianProcessClassifier()
  if model_name =="lda":
    clf=LinearDiscriminantAnalysis()
  if model_name =="qda":
    clf=QuadraticDiscriminantAnalysis() 
  return clf

In [ ]:
from sklearn.metrics import f1_score
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from modAL.uncertainty import uncertainty_sampling
from functools import partial
from keras.wrappers.scikit_learn import KerasClassifier
import random
from sklearn.model_selection import train_test_split
from collections import Counter
#data_train,train_labels
random_seed_list=[5,12,42,29,54]
def run_multy_sklearn(BATCH_SIZE,data_test,test_labels,num_of_run,model_name,strategy,train_size,is_random):
  # counter_random = 5
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)

  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0
  for i in range(num_of_run):
    clf = get_model(model_name)
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    
    learner = ActiveLearner(
    estimator=clf,
    query_strategy=preset_batch,
    X_training=X_train, y_training=y_train)
    t1 = time.time()


    # Allow our model to query our unlabeled dataset for the most
    # informative points according to our query strategy (uncertainty sampling).
    # counter_random= 1 
    # calculate initial batch
    y_pred = learner.predict(X_test)
    macro=f1_score(y_test,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro
    # calculate initial batch majority
    counts = np.bincount(y_train)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test))]
    macro=f1_score(y_test,majority, average='macro')
    pref_hist_majority[i][0]=macro
    #calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test))]
    macro=f1_score( y_test,y_rand, average='macro')
    pref_random = pref_random + macro
    #active leraning loop
    for index in range(1,N_QUERIES+1):
      # counter_random += 1 
      t2 = time.time()
      print(i,'th run and query number',index)
      query_index, query_instance = learner.query(X_pool)
      print('num if query',len(query_index))
      if is_random : #or counter_random % random_ratio == 0:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)

      # Teach our ActiveLearner model the record it has requested.
    
      X, y = X_pool[query_index], y_pool[query_index]
      for j in range(1):
        learner.teach(X=X, y=y)
      # Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)

      # Calculate and report our model's accuracy.
      model_accuracy = learner.score(X_test, y_test)
      y_pred = learner.predict(X_test)
      


      macro=f1_score( y_test,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))
      # Save our model's performance for plotting.
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro
      # calculate and add majority
      counts = np.bincount(y_train)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test))]
      macro=f1_score(y_test,majority, average='macro')
      pref_hist_majority[i][index]=macro
      print(format_time(time.time()-t2))
    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)

  return pref_hist_multy_acc_avg,pref_hist_multy_f1_avg,pref_random,pref_hist_majority_avg #,pref_hist_multy_accuracy,pref_hist_multy_f1

### Run_multy (for pytorch MLP)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from modAL.models import ActiveLearner
from modAL.batch import uncertainty_batch_sampling
from functools import partial
import random
random_seed_list=[5,12,42,29,54]
def run_multy(BATCH_SIZE,data_test,test_labels,num_of_run,model,strategy,train_size,is_random):
  test_pool_split=0.5
  preset_batch = partial(strategy, n_instances=BATCH_SIZE)
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(data_test,test_labels, test_size=train_size,shuffle=True )
  X_pool_0, X_test_0, y_pool_0, y_test_0 = train_test_split(X_pool_0,y_pool_0, test_size=test_pool_split,shuffle=True )
  N_QUERIES = int(len(X_pool_0)/BATCH_SIZE)
  print(num_of_run,N_QUERIES)
  pref_hist_multy_accuracy=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_multy_f1=np.zeros((num_of_run,N_QUERIES+1))
  pref_hist_majority = np.zeros((num_of_run,N_QUERIES+1))
  pref_random = 0

  test_labels = keras.utils.to_categorical(test_labels, 2)  
  for i in range(num_of_run):
    X_pool, X_train, y_pool, y_train = train_test_split(data_test, test_labels, test_size=train_size,shuffle=True,random_state=random_seed_list[i])
    X_pool, X_test, y_pool, y_test = train_test_split(X_pool,y_pool, test_size=test_pool_split,shuffle=True,random_state=random_seed_list[i])
    clf = KerasClassifier(model)
    print('x pool length',X_train.shape)
    print('y pool length',y_train.shape)
    learner = ActiveLearner(
    estimator=clf,
    query_strategy=preset_batch,
    X_training=X_train, y_training=y_train)

    t1 = time.time()

    # Allow our model to query our unlabeled dataset for the most
    # informative points according to our query strategy (uncertainty sampling).

    # calculate initial batch
    y_pred = learner.predict(X_test)
    y_test_cat = np.argmax(y_test,axis=1)
    macro=f1_score(y_test_cat,y_pred, average='macro')
    pref_hist_multy_f1[i][0]=macro
    # calculate initial batch majority
    y_train_cat = np.argmax(y_test,axis=1)
    counts = np.bincount(y_train_cat)
    value = np.argmax(counts)
    majority =[value for i in range(len(y_test_cat))]
    macro=f1_score(y_test_cat,majority, average='macro')
    pref_hist_majority[i][0]=macro
    #calculate random
    y_rand=[random.randint(0,2) for i in range(len(y_test_cat))]
    macro=f1_score( y_test_cat,y_rand, average='macro')
    pref_random = pref_random + macro
    for index in range(1,N_QUERIES+1):

      query_index, query_instance = learner.query(X_pool)
      # print('num if query',len(query_index))
      if is_random:
        index_list = range(len(X_pool))
        query_index = random.sample(index_list,BATCH_SIZE)
      # Teach our ActiveLearner model the record it has requested.

      X, y = X_pool[query_index], y_pool[query_index]

      for j in range(5):
        learner.teach(X=X, y=y)
      # Remove the queried instance from the unlabeled pool.
      X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index,axis=0)
      print('sec',y_pool.shape)
      # Calculate and report our model's accuracy.
      model_accuracy = learner.score(data_test, test_labels)
      y_pred = learner.predict(X_test)
      
      # y_pred_cat=np.argmax(y_pred)
      y_test_cat=np.argmax(y_test,axis=1)
      # print('shapeè',y_test_cat.shape)
      macro=f1_score( y_test_cat,y_pred, average='macro')
      print('after query {n}: Accuracy :{acc:0.4f} macro f1 :{f1:0.4f}'.format(n=index + 1, acc=model_accuracy,f1=macro))
      # Save our model's performance for plotting.
      pref_hist_multy_accuracy[i][index]=model_accuracy
      pref_hist_multy_f1[i][index]=macro

       # calculate and add majority
      y_train_cat = np.argmax(y_test,axis=1)
      counts = np.bincount(y_train_cat)
      value = np.argmax(counts)
      majority =[value for i in range(len(y_test_cat))]
      macro=f1_score(y_test_cat,majority, average='macro')
      pref_hist_majority[i][index]=macro
      
    print(format_time(time.time()-t1))
  pref_random= pref_random/num_of_run
  pref_hist_majority_avg=pref_hist_majority.mean(0)
  pref_hist_multy_acc_avg=pref_hist_multy_accuracy.mean(0)
  pref_hist_multy_f1_avg=pref_hist_multy_f1.mean(0)
  return pref_hist_multy_acc_avg,pref_hist_multy_f1_avg ,pref_random,pref_hist_majority_avg#,pref_hist_multy_accuracy,pref_hist_multy_f1
    

# Automated Run

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
clf_rf=RandomForestClassifier(max_depth=1000, random_state=0)
clf_Ada = AdaBoostClassifier()
clf_svm = svm.SVC(probability=True)
sentence_feature_normal=np.load(DATA_PATH+'/vectorization/normal_bert.npy')
sentence_feature_tweet =np.load(DATA_PATH+'/vectorization/tweet_bert.npy')
sentence_feature_GLOVE =np.load(DATA_PATH+'/vectorization/GLOVE.np')
model_dict={
    "rf":clf_rf,
    "Ada":clf_Ada,
    "svm":clf_svm,
    # "mlp":create_keras_model
}
reps_dict={
    "BERT":sentence_feature_normal,
    "tweetBERT":sentence_feature_tweet,
    "GLOVE":sentence_feature_GLOVE,
}

In [ ]:
number_of_run = 5
def AL_loop(rep,model,AL_stg):
  sentence_feature=reps_dict[rep]
  data_test = sentence_feature[train_start:train_end+1]
  if AL_stg=="lc": stg=False
  if AL_stg=="Random": stg=True
  if model=="mlp":
    if rep!= "GLOVE":
      acc_avg,f1_avg,pref_random,pref_hist_majority = run_multy(50, data_test, test_labels,number_of_run,create_keras_model_berts,uncertainty_batch_sampling,20,stg)
    else:
      acc_avg,f1_avg,pref_random,pref_hist_majority = run_multy(50, data_test, test_labels,number_of_run,create_keras_model_GLOVE,uncertainty_batch_sampling,20,stg)  
  else:
      acc_avg,f1_avg,pref_random,pref_hist_majority = run_multy_sklearn(50,data_test, test_labels,number_of_run,model,uncertainty_sampling,20,stg)
  return acc_avg,f1_avg,pref_random,pref_hist_majority 

In [ ]:
from google.colab import files
counter = 0

for rep in reps:
  for model in models:
    for AL_stg in stratefies:
      counter = counter+1
      acc_avg,f1_avg,pref_random,pref_hist_majority  = AL_loop(rep,model,AL_stg)
      output_addr_rand=rep+"-"+model+"-"+AL_stg+'-'+str(topic_num)+"-random_baseline.np"
      output_addr_majority=rep+"-"+model+"-"+AL_stg+'-'+str(topic_num)+"-majority_baseline.np"
      output_addr_f1=rep+"-"+model+"-"+AL_stg+'-'+str(topic_num)+"-f1.np"
      output_addr_accuracy=rep+"-"+model+"-"+AL_stg+'-'+str(topic_num)+"-accuracy.np"
      save_result(np.array(pref_random),output_addr_rand)
      save_result(np.array(pref_hist_majority),output_addr_majority)
      save_result(np.array(f1_avg),output_addr_f1)
      save_result(np.array(acc_avg),output_addr_accuracy)
      files.download("/content/"+output_addr_f1)
      # files.download("/content/"+output_addr_accuracy)    
      files.download("/content/"+output_addr_rand)
      files.download("/content/"+output_addr_majority)      

0 th run and query number 1
num if query 50
after query 2: Accuracy :0.7491 macro f1 :0.6231
0:00:00
0 th run and query number 2
num if query 50
after query 3: Accuracy :0.7705 macro f1 :0.6551
0:00:00
0 th run and query number 3
num if query 50
after query 4: Accuracy :0.7811 macro f1 :0.6605
0:00:00
0 th run and query number 4
num if query 50
after query 5: Accuracy :0.7740 macro f1 :0.6605
0:00:00
0 th run and query number 5
num if query 50
after query 6: Accuracy :0.7544 macro f1 :0.6702
0:00:00
0 th run and query number 6
num if query 50
after query 7: Accuracy :0.7811 macro f1 :0.7025
0:00:00
0 th run and query number 7
num if query 50
after query 8: Accuracy :0.7883 macro f1 :0.7203
0:00:00
0 th run and query number 8
num if query 50
after query 9: Accuracy :0.7989 macro f1 :0.7280
0:00:00
0 th run and query number 9
num if query 50
after query 10: Accuracy :0.7989 macro f1 :0.7332
0:00:00
0 th run and query number 10
num if query 50
after query 11: Accuracy :0.8025 macro f1 :0.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
f1_avg

array([0.49038117, 0.45644872, 0.38471042, 0.32915433, 0.33053905,
       0.27040931, 0.25984697, 0.26512427, 0.26735544, 0.24342608,
       0.25803875, 0.24497486])